In [1]:
### RUN EVERY TIME: COMTRADE DATASETS EXTRACTING

In [2]:
### RUN EVERY TIME: INITIALIZATION

import pandas as pd
import numpy as np
import gc
import os
import datetime
import time
import networkx as nx
import matplotlib.pyplot as plt

In [3]:
### RUN EVERY TIME: VERSION CONTROL

from platform import python_version
print('python version: ', python_version())
print('numpy version: ', np.__version__)
print('pandas version: ', pd.__version__)

python version:  3.7.4
numpy version:  1.17.2
pandas version:  0.25.3


In [4]:
### RUN EVERY TIME: MAIN CONSTANTS

### MultiIndex level slice constant:
All = slice(None)
### Universe path:
str_path_universe = 'Data_Files/Source_Files/acadian_universe.xlsx'
### Dates:
str_date_end = '2022-12-31'
date_start = pd.Timestamp('1989-12-29')
date_end = pd.Timestamp(str_date_end)
date_ison = pd.Timestamp('1994-12-31')
### NA for MS Excel files:
list_na_excel_values = ['', '#N/A', '#N/A N/A', '#NA', '-1.#IND', '-1.#QNAN', '-NaN', '-nan', '1.#IND', '1.#QNAN', 'N/A', 'NULL', 'NaN', 'n/a', 'nan', 'null',
                        '#N/A Requesting Data...', '#N/A Invalid Security', '#N/A Field Not Applicable', '---']
### Checked EBOPS service IDs list (df_serv_to_gics['GICS Group Code']):
list_services = ['206', '210', '214', '218', '219', '223', '227', '231', '232', '237', '240', '246', '247', '250', '251', '254', '255', '256', '257', '258', '263',
                 '264', '269', '272', '273', '288', '289', '292', '293', '294', '310', '391', '431', '500', '888', '891', '892', '894', '950']
### Augmented bilateral export:
str_path_export_bilateral = 'Data_Files/Source_Files/comtrade_export_bilateral.h5'
### Export key:
str_key_unc_export = 'export_augmented'
### Augmented bilateral import:
str_path_import_bilateral = 'Data_Files/Source_Files/comtrade_import_bilateral.h5'
### Import key:
str_key_unc_import = 'import_augmented'
### Augmented bilateral import:
str_path_gdp = 'Data_Files/Source_Files/gdp_dataset.h5'
### Factor file:
str_path_simple_exp = 'Data_Files/Source_Files/comtrade_simple_exp.h5'
str_path_pagerank_exp = 'Data_Files/Source_Files/comtrade_pagerank_exp.h5'
str_key_comtrade_factor = 'comtrade_factor'
str_path_factor_xlsx = 'Data_Files/Source_Files/comtrade_factor.xlsx'
### IOnteraction variable file:
str_path_interact_var = 'Data_Files/Source_Files/comtrade_interact_exp.h5'
str_key_interact_var = 'interact_var'

In [5]:
### DEFINING WEIGHTED AVERAGE CALCULATOR

def weighted_average(ser_data, ser_weight = None, int_min_count = 0):
    ### Default output:
    num_result = np.NaN
    ### Checking for data presence:
    if (ser_data.count() > int_min_count):       
        ### Checking for weights dataset:
        if ser_weight is None:
            ### Calculating of simple average:
            num_result = np.nanmean(ser_data.values)
        else:
            ### Weights filtering:
            list_weight = ser_weight[ser_data.dropna().index].values
            ### Checking for weights presence:
            if np.nansum(list_weight):
                ### Data filtering:
                list_data = ser_data.dropna().values
                ### Weighted average calculating:
                num_result = np.nansum(list_data * list_weight) / np.nansum(list_weight)
    ### Results output:
    return num_result

In [6]:
### DEFINING COUNTRY CODES EXTRACTOR

def get_country_codes(use_local_copy = False):  
    ### In case if URL is unavailable:
    if (use_local_copy):
        url_country_code = 'Data_Files/Source_Files/countrycode.html'
    ### Online extraction:
    else:
        url_country_code = 'https://countrycode.org/'
    df_full_codes = pd.read_html(url_country_code, index_col = 'COUNTRY')[0]
    df_full_codes[['ISO SHORT', 'ISO LONG']] = df_full_codes['ISO CODES'].str.split(' / ', expand = True)
    df_result = df_full_codes[['ISO SHORT', 'ISO LONG']].sort_index()    
    df_result.index = df_result.index.str.upper()
    ### Results output:
    return df_result

In [7]:
### DEFINING EXTRACTION UNIVERSE DATA FROM MS EXCEL SOURCE (TO BE IGNORED IN PRODUCT CODE)

def ison_membership_converting(str_path_universe, date_end, bool_daily = False, int_backfill_months = 0):
    ### Defining business-month-end reindexation on country level:
    def country_modify(ser_raw_country, date_end):
        ser_res_country = ser_raw_country.droplevel(0).resample('MS').last().resample('BM').last()
        range_country = pd.date_range(ser_res_country.index[0], date_end, freq = 'BM')
        return ser_res_country.reindex(range_country).ffill()
    ### Markets encoding table:
    dict_markets = {50 : 'DM', 57 : 'EM', 504 : 'FM', 0: np.NaN}     
    ### Loading source file:
    df_raw_universe = pd.read_excel(engine = 'openpyxl', io = str_path_universe, sheet_name = 'Switchers', header = 0, parse_dates = True, index_col = [0, 1],
                                 na_values = ['', '#N/A', '#N/A N/A', '#NA', '-1.#IND', '-1.#QNAN', '-NaN', '-nan', '1.#IND', 
                                             '1.#QNAN', 'N/A', 'NULL', 'NaN', 'n/a', 'nan', 'null'], keep_default_na = False)
    ### Converting source file:
    df_raw_universe.index.names = ['Country', 'Date']
    ser_raw_universe = df_raw_universe['Region']
    ser_raw_universe.fillna(0, inplace = True)
    ser_raw_universe.name = 'Market'
    ### By country reindexation and translation:
    ser_res_universe = ser_raw_universe.groupby('Country').apply(country_modify, date_end)
    ser_res_universe.index.names = ['Country', 'Date']
    ser_res_universe = ser_res_universe.replace(dict_markets).reorder_levels([1, 0]).sort_index() 
    ### Expanding membership for primary regions members by backfilling:
    if int_backfill_months:
        ### List of regions:
        list_region = list(ser_res_universe.dropna().unique())
        ### Initialising of collection of series with backfilled data for each region:
        list_ison_backfill = []
        ### Regions looping:
        for iter_region in list_region:
            ### Defining start of region date:
            date_first_valid = ser_res_universe.loc[ser_res_universe == iter_region].first_valid_index()[0]
            ### Creating dates index to backfilling:
            idx_date_backfill = pd.date_range(end = date_first_valid, periods = int_backfill_months + 1, freq = 'BM')[: -1]
            ### Creating primary countries index to backfilling:            
            idx_region_backfill = ser_res_universe.loc[ser_res_universe == iter_region].loc[date_first_valid, All].index.get_level_values('Country')
            ### Creating full index:
            idx_ison_backfill = pd.MultiIndex.from_product([idx_date_backfill, idx_region_backfill])
            ### Series with backfilled data:
            list_ison_backfill.append(pd.Series(iter_region, index = idx_ison_backfill))
        ### Combination of backfilled series and original ISON data:    
        ser_res_universe = ser_res_universe.combine_first(pd.concat(list_ison_backfill, axis = 0)).sort_index()  
        ser_res_universe.index.names = ['Date', 'Country']
    ### Converting to daily frequency:
    if bool_daily:
        ser_res_universe = ser_res_universe.reset_index('Country').groupby('Country').resample('B').ffill()['Market'].swaplevel().sort_index()    
    ### Results output:
    ser_res_universe.name = 'Market'
    return ser_res_universe

In [8]:
### RUN EVERY TIME: COMMON DATA EXTRACTION STEPS

### World Country Codes:
df_country_codes = get_country_codes()
### ISON membership history:
ser_ison_membership = ison_membership_converting(str_path_universe, pd.to_datetime(str_date_end))
ser_ison_membership.index.names = ['Date', 'Reporter']
### ISON Members:
list_ison_countries = sorted(ser_ison_membership.index.get_level_values('Reporter').unique())
### ISON status for the last available date:
ser_ison_status = ser_ison_membership.loc[ser_ison_membership.index[-1][0]]

In [ ]:
### BILATERAL EXPORT DATA LOADING TO PERFORM FACTOR CALCULATION

gc.collect()
list_export_chunks = []
for num_iter_number, ser_iter_chunk in enumerate(pd.read_hdf(str_path_export_bilateral, key = str_key_unc_export, chunksize = 1000000)):
    gc.collect()
    print(num_iter_number, ': Extraction started')
    ser_iter_chunk = ser_iter_chunk[ser_iter_chunk > 0.0].astype('int32')
    df_iter_chunk = ser_iter_chunk.reset_index()
    df_iter_chunk = df_iter_chunk[(df_iter_chunk['Reporter'] != df_iter_chunk['Partner']) & \
                                  ((df_iter_chunk['Type'] == 'Goods') | df_iter_chunk['Commodity_ID'].isin(list_services)) & (df_iter_chunk['Reporter'] != 'World') & \
                                  (df_iter_chunk['Partner'] != 'World')]\
                               .drop('Type', axis = 1)    
    print(num_iter_number, ': Filtering performed')    
    ser_iter_chunk = df_iter_chunk.set_index(['Date', 'Reporter', 'Partner', 'Commodity_ID']).squeeze().sort_index()
    del df_iter_chunk
    gc.collect()
    list_export_chunks.append(ser_iter_chunk)
    print(num_iter_number, ': Chunk added to container')    
ser_bilateral_export = pd.concat(list_export_chunks, axis = 0, sort = False).sort_index()
ser_bilateral_export.name = 'Export'
del list_export_chunks
gc.collect()

In [ ]:
### BILATERAL IMPORT DATA LOADING TO PERFORM FACTOR CALCULATION

gc.collect()
list_import_chunks = []
for num_iter_number, ser_iter_chunk in enumerate(pd.read_hdf(str_path_import_bilateral, key = str_key_unc_import, chunksize = 1000000)):
    gc.collect()
    print(num_iter_number, ': Extraction started')
    ser_iter_chunk = ser_iter_chunk[ser_iter_chunk > 0.0].astype('int32')
    df_iter_chunk = ser_iter_chunk.reset_index()
    df_iter_chunk = df_iter_chunk[(df_iter_chunk['Reporter'] != df_iter_chunk['Partner']) & \
                                  ((df_iter_chunk['Type'] == 'Goods') | df_iter_chunk['Commodity_ID'].isin(list_services)) & (df_iter_chunk['Reporter'] != 'World') & \
                                  (df_iter_chunk['Partner'] != 'World')]\
                               .drop('Type', axis = 1)     
    print(num_iter_number, ': Filtering performed')    
    ser_iter_chunk = df_iter_chunk.set_index(['Date', 'Reporter', 'Partner', 'Commodity_ID']).squeeze().sort_index()
    del df_iter_chunk
    gc.collect()
    list_import_chunks.append(ser_iter_chunk)
    print(num_iter_number, ': Chunk added to container')    
ser_bilateral_import = pd.concat(list_import_chunks, axis = 0, sort = False).sort_index()
ser_bilateral_import.name = 'Import'
del list_import_chunks
gc.collect()

In [11]:
### REPORTER / COMMODITY BY DATE TOTAL EXPORT & IMPORT & TRADE

gc.collect()
### Export totals:
ser_country_comm_export = ser_bilateral_export.groupby(['Date', 'Reporter', 'Commodity_ID']).sum().dropna()
ser_country_comm_export.name = 'Export'
### Import totals:
ser_country_comm_import = ser_bilateral_import.groupby(['Date', 'Reporter', 'Commodity_ID']).sum().dropna()
ser_country_comm_import.name = 'Import'
### Adding trade totals:
df_country_comm_trade = pd.concat([ser_country_comm_export, ser_country_comm_import], axis = 1)
df_country_comm_trade = df_country_comm_trade.unstack('Date').stack('Date', dropna = False)
df_country_comm_trade = df_country_comm_trade.unstack('Reporter').stack('Reporter', dropna = False)
df_country_comm_trade = df_country_comm_trade.unstack('Commodity_ID').stack('Commodity_ID', dropna = False).fillna(0.0)
df_country_comm_trade['Trade'] = df_country_comm_trade['Export'] + df_country_comm_trade['Import']

In [12]:
### TOTAL COMMODITY BY DATE TRADE VOLUME & CROSS-SECTIONAL WEIGHT

### Total commodity volume:
ser_commodity_trade = df_country_comm_trade.groupby(['Date', 'Commodity_ID'])['Trade'].sum().swaplevel().sort_index()
ser_commodity_trade.name = 'Commodity_Total'
df_commodity_trade = ser_commodity_trade.to_frame()
### Total commodity weight:
df_commodity_trade['Commodity_Weight'] = df_commodity_trade['Commodity_Total'].groupby('Date').transform(lambda df_group: df_group / df_group.sum())
### Resampling to monthly data:
def reindex_monthly(df_group):
    df_result = df_group.droplevel(['Commodity_ID']).reindex(pd.date_range(df_group.index[0][-1], str_date_end, freq = 'BY'))
    df_result = df_result.resample('BM').ffill()
    return df_result
df_commodity_trade = df_commodity_trade.groupby('Commodity_ID').apply(reindex_monthly)
df_commodity_trade.index.names = ['Commodity_ID', 'Date']

In [13]:
### REPORTER / COMMODITY BY DATE EXPORT WEIGHT

gc.collect()
ser_export_weight = ser_country_comm_export.groupby(['Date', 'Commodity_ID']).transform(lambda ser_group: ser_group / ser_group.sum())
ser_export_weight.name = 'Weight_Local'

In [14]:
### REPORTER / COMMODITY BY DATE PAGE RANK


gc.collect()
def get_pagerank(df_group):
    nx_graph = nx.from_pandas_edgelist(df_group, 'Reporter', 'Partner', edge_attr = 'Export', create_using = nx.DiGraph)
    dict_pagerank = nx.pagerank(nx_graph)
    ser_pagerank = pd.Series(dict_pagerank)
    return ser_pagerank
    
ser_export_pagerank = ser_bilateral_export.reset_index().groupby(['Date', 'Commodity_ID']).apply(get_pagerank)
ser_export_pagerank.name = 'PG_Rank_Local'
ser_export_pagerank.index.names = ['Date', 'Commodity_ID', 'Reporter']
ser_export_pagerank = ser_export_pagerank.reorder_levels(['Date', 'Reporter', 'Commodity_ID']).sort_index()

In [49]:
### TEMP

str_beg_date = '1999-12-31'
str_end_date = '2000-12-29'
str_service_id = '240'

ser_test_exp = ser_bilateral_export.loc[str_end_date, :, :, str_service_id].droplevel(['Date', 'Commodity_ID'])
#nx.from_pandas_edgelist(ser_test_exp.reset_index(), 'Reporter', 'Partner', edge_attr = 'Export', create_using = nx.DiGraph)
ser_test_pagerank = get_pagerank(ser_test_exp.reset_index())
ser_test_pagerank.name = 'PG_Rank'
ser_test_simple = ser_test_exp.groupby('Reporter').sum()
ser_test_simple = ser_test_simple / ser_test_simple.sum()
ser_test_simple.name = 'Simple_Weight'

df_test_both = pd.concat([ser_test_pagerank, ser_test_simple], axis = 1, sort = True)
df_test_both.corr()
#df_test_both.to_excel('Data_Files/Test_Files/weights_comparision.xlsx')

,PG_Rank,Simple_Weight
PG_Rank,1.000000,0.088508
Simple_Weight,0.088508,1.000000


In [68]:
### TEMP

#ser_test_exp.groupby('Partner').count()
display(ser_test_exp.loc[['IT']])
display(ser_test_exp.loc[:, ['IT']])
display(ser_test_exp.loc[['RS']])
display(ser_test_exp.loc[:, ['RS']])

Reporter  Partner
IT        CA          209362
          JP         1067283
Name: Export, dtype: int32

Reporter  Partner
CA        IT         66570
JP        IT         13600
Name: Export, dtype: int32

Reporter  Partner
RS        AL             9
          AT           213
          AU           127
          BA           814
          BE            70
          CA           110
          CY           940
          CZ           191
          DE         15118
          DK            19
          ES            10
          FI             3
          GB           894
          GR            34
          HR            77
          HU            21
          JP             2
          NL            64
          NO             9
          RO             2
          RU           152
          SE            53
          SI           165
          SK            84
          VA           438
          VI          2188
          VN             4
Name: Export, dtype: int32

Reporter  Partner
AL        RS           24
AT        RS         1197
AU        RS            6
BA        RS           13
BE        RS           81
BG        RS          230
BY        RS          385
CA        RS            1
CN        RS          141
CU        RS            0
CY        RS         4495
CZ        RS            2
DE        RS         4424
DK        RS            5
DO        RS            1
EG        RS            5
ES        RS            8
FI        RS           17
GB        RS         7192
GR        RS          237
HR        RS         1599
HU        RS          523
IE        RS          304
IL        RS           18
IN        RS            0
JO        RS          122
KM        RS            6
LK        RS           13
LU        RS           94
MD        RS            2
MH        RS            5
MT        RS           22
NL        RS           13
NO        RS           36
PA        RS           30
PL        RS            2
RO        RS           88
RU        RS        

In [15]:
### TEMP

df_export_both = pd.concat([ser_export_weight, ser_export_pagerank], axis = 1)



df_beg_both = df_export_both.loc[(str_beg_date, ser_ison_status.index, All), :].droplevel('Date')
df_beg_both = df_beg_both.join(df_country_comm_trade.loc[str_beg_date].droplevel('Date')['Trade'])
df_end_both = df_export_both.loc[(str_end_date, ser_ison_status.index, All), :].droplevel('Date')
df_end_both = df_end_both.join(df_country_comm_trade.loc[str_end_date].droplevel('Date')['Trade'])

In [16]:
### TEMP

ser_beg_simple = df_beg_both.groupby('Reporter', group_keys = False).apply(lambda df_group: weighted_average(df_group['Weight_Local'], df_group['Trade']))
ser_beg_pagerank = df_beg_both.groupby('Reporter', group_keys = False).apply(lambda df_group: weighted_average(df_group['PG_Rank_Local'], df_group['Trade']))
ser_end_simple = df_end_both.groupby('Reporter', group_keys = False).apply(lambda df_group: weighted_average(df_group['Weight_Local'], df_group['Trade']))
ser_end_pagerank = df_end_both.groupby('Reporter', group_keys = False).apply(lambda df_group: weighted_average(df_group['PG_Rank_Local'], df_group['Trade']))
df_simple_mean = pd.read_hdf(str_path_simple_exp, str_key_comtrade_factor)
df_pg_rank_mean = pd.read_hdf(str_path_pagerank_exp, str_key_comtrade_factor)
display(df_simple_mean.loc[(All, str_beg_date), 'Weight_Local'].droplevel('Date').equals(ser_beg_simple))
display(df_pg_rank_mean.loc[(All, str_beg_date), 'PG_Rank_Local'].droplevel('Date').equals(ser_beg_pagerank))
display(df_simple_mean.loc[(All, str_end_date), 'Weight_Local'].droplevel('Date').equals(ser_end_simple))
display(df_pg_rank_mean.loc[(All, str_end_date), 'PG_Rank_Local'].droplevel('Date').equals(ser_end_pagerank))
display(ser_beg_simple.corr(ser_beg_pagerank))
display(ser_end_simple.corr(ser_end_pagerank))
display(ser_beg_simple.corr(ser_end_simple))
display(ser_beg_pagerank.corr(ser_end_pagerank))

True

True

True

True

0.8080087763985896

0.1993989668947142

0.9828768071853787

0.9310211979562197

In [77]:
### TEMP

#display(pd.concat([df_beg_both.groupby('Reporter')['Weight_Local'].count(), df_end_both.groupby('Reporter')['Weight_Local'].count()], axis = 1, sort = True))
set(df_end_both.loc['AU'].index) - set(df_beg_both.loc['AU'].index)

{'206',
 '210',
 '214',
 '219',
 '223',
 '232',
 '237',
 '240',
 '246',
 '247',
 '250',
 '251',
 '255',
 '256',
 '257',
 '263',
 '264',
 '269',
 '272',
 '273',
 '288',
 '289',
 '292',
 '293',
 '294',
 '891',
 '892'}

In [ ]:
### TEMP

'''
1) Check autocorrelation
2) Check commodities count
3) Check reporters coverage
'''